In [3]:
from math import (atan2, pi, sqrt)    

from collections import OrderedDict

import os
import numpy as np
import torch
import torch.nn.functional as F

torch.set_printoptions(sci_mode=False)

from src.utilities.polar_traversal import (
    Bridge,
    make_latitudes,
    get_start,
    gather_path,
    cumulative_distances,
)

import src.utilities.polar_traversal as polar_traversal


def get_dist(x, y):
    return sqrt((x-0.5) **2 + (y-0.5) ** 2)

def get_angle(x, y, square_no):
    side = (square_no + 1) * 2
    corr = (side -1) / 2    
    angle = atan2(y -corr, x -corr)
    return angle if angle >= 0 else angle + 2 * pi


def get_sample(pos, normed, path_points, path_normals):    
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)

def single_path_sample(path, layer_no, total_layers):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    pos = (layer_no+1) / (total_layers+1)
    pnt, nrm, _ = get_sample(pos, normed, 
                             path['points'], path['normals'])
    return pnt, nrm

def get_angles_series(n):
    return [x for x in range(0, n*2+1, 2)]


def get_sample(pos, normed, path_points, path_normals):    
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)

def single_path_sample(path, layer_no, total_layers):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    pos = layer_no / total_layers    
    pnt, nrm, _ = get_sample(pos, normed, 
                             path['points'], path['normals'])
    return torch.tensor(pnt).float(), torch.tensor(nrm).float()

def square_indices(n):
    max_idx = 2 * (n+1)
    res = [[0, x] for x in range(max_idx -1, -1, -1)]\
        + [[x, 0] for x in range(1, max_idx)]\
        + [[max_idx -1, x] for x in range(1, max_idx)]\
        + [[x, max_idx -1] for x in range(max_idx -2, 0, -1)]
    return res

def shifted_square_indices(sq_no, square_no):    
    sq_ids =  square_indices(sq_no)
    ost = square_no - sq_no
    return [(i+ost, k+ost) for (i, k) in iter(sq_ids)]

def get_latitudes_angles(n):
    return (torch.arange(0, n) * (2 * pi / n)).tolist()

class Blueprint:
    
    def __init__(self, path, latitudes_num):
        self.path = path
        self.latitudes_num = latitudes_num
        self.bridge = Bridge(path)
        paths_path =  f'./data/paths_{latitudes_num}.pth'        
        if os.path.exists(paths_path):
            self.paths = torch.load(paths_path) 
        else:
            self.paths = self.gather_paths(latitudes_num)
        
    
    def gather_paths(self, latitudes_num=None):
        latitudes_num = latitudes_num or self.latitudes_num  
        latitude_angles = get_latitudes_angles(latitudes_num)        
        latitudes = make_latitudes(latitudes_num)
        bridge = self.bridge
        face_id, point, normal = get_start(bridge.mesh)
        paths = OrderedDict([])
        for i, (latitude, latitude_angle) in enumerate(zip(latitudes, latitude_angles)):
            print(i, latitudes_num)
            path = gather_path(latitude, face_id, point, normal, bridge)
            paths[latitude_angle] = path
        return paths
        
    def __repr__(self):
        return f'Blueprint: {self.path} latitudes_num: {self.latitudes_num}' 

    def get_border_paths(self, angle):        
        assert angle >=0 and angle < 2 * pi
        paths = self.paths
        angle_keys = list(paths.keys())
        for i, angle_key in enumerate(angle_keys):        
            if angle < angle_key:            
                return (paths[angle_keys[i % len(angle_keys)]],
                        paths[angle_keys[(i+1) % len(angle_keys)]])
        return (paths[angle_keys[0]], paths[angle_keys[-1]])

    def grid_sample(self, square_no, save_path=None):
        side = (square_no + 1) * 2
        res = self.load_result(save_path) \
            if save_path is not None and os.path.exists(save_path) \
            else self.create_result(side)
        
        face_id, point, normal = get_start(self.bridge.mesh)
        for sq_no in range(square_no+1):
            print(sq_no, res['sq_no'][0])
            if sq_no > res['sq_no'][0]:                                
                side = (sq_no + 1) * 2
                sq_ids = shifted_square_indices(sq_no, square_no)            
                angles_no = len(sq_ids)                          
                for (r, c) in sq_ids:
                    angle = get_angle(r, c, square_no)                    
                    path0, path1 = self.get_border_paths(angle)
                    pnt0, nrm0 = single_path_sample(path0, sq_no, square_no)
                    pnt1, nrm1 = single_path_sample(path1, sq_no, square_no)
                    pnt = (pnt0 + pnt1)/2
                    nrm = F.normalize((nrm0 + nrm1)/2, dim=0)
                    res['points'][0, :, c, r] = pnt
                    res['normals'][0, :, c, r] = nrm                
                    res['grid'][c, r] = float(sq_no)                
                res['sq_no'][0] = sq_no
                self.save_result(res, save_path)
        return res            
        
    def create_result(self, side):
        return {
            'points': torch.zeros(1, 3, side, side),
            'normals': torch.zeros(1, 3, side, side),
            'grid': torch.zeros(side, side) - 9,
            'sq_no': torch.tensor([-1]),
        }
    
    def load_result(self, path):
        loaded = np.load(path)
        
        return {
            'points': torch.tensor(loaded['points']).float(),
            'normals': torch.tensor(loaded['normals']).float(),
            'grid': torch.tensor(loaded['grid']).float(),
            'sq_no': torch.tensor(loaded['sq_no']).long(),
        }
    
    def save_result(self, res, path=None):
        #assert type('s')  == 'str'
        if path is not None:            
            print('path', path, res['sq_no'])
            np.savez_compressed(
                path,
                points=res['points'].numpy(),
                normals=res['normals'].numpy(),
                grid=res['grid'].numpy(),
                sq_no=res['sq_no'].numpy(),
            )
        

#stl_path = './data/vezuvio.stl'
#stl_path = './data/plane.stl'
stl_path = './data/circle.stl'

square_no = 255
blueprint = Blueprint(stl_path, 2048)
print(blueprint)
torch.save(blueprint.paths, f'./data/circle_plane_{len(blueprint.paths)}.pth')
blueprint_path = './data/circle_{}.npz'.format(square_no)
# print(blueprint_path)
res = blueprint.grid_sample(square_no, blueprint_path)
res['points'].shape, res['normals'].shape

0 2048
1 2048
2 2048
3 2048
4 2048
5 2048
6 2048
7 2048
8 2048
9 2048
10 2048
11 2048
12 2048
13 2048
14 2048
15 2048
16 2048
17 2048
18 2048
19 2048
20 2048
21 2048
22 2048
23 2048
24 2048
25 2048
26 2048
27 2048
28 2048
29 2048
30 2048
31 2048
32 2048
33 2048
34 2048
35 2048
36 2048
37 2048
38 2048
39 2048
40 2048
41 2048
42 2048
43 2048
44 2048
45 2048
46 2048
47 2048
48 2048
49 2048
50 2048
51 2048
52 2048
53 2048
54 2048
55 2048
56 2048
57 2048
58 2048
59 2048
60 2048
61 2048
62 2048
63 2048
64 2048
65 2048
66 2048
67 2048
68 2048
69 2048
70 2048
71 2048
72 2048
73 2048
74 2048
75 2048
76 2048
77 2048
78 2048
79 2048
80 2048
81 2048
82 2048
83 2048
84 2048
85 2048
86 2048
87 2048
88 2048
89 2048
90 2048
91 2048
92 2048
93 2048
94 2048
95 2048
96 2048
97 2048
98 2048
99 2048
100 2048
101 2048
102 2048
103 2048
104 2048
105 2048
106 2048
107 2048
108 2048
109 2048
110 2048
111 2048
112 2048
113 2048
114 2048
115 2048
116 2048
117 2048
118 2048
119 2048
120 2048
121 2048
122 2048
123

926 2048
927 2048
928 2048
929 2048
930 2048
931 2048
932 2048
933 2048
934 2048
935 2048
936 2048
937 2048
938 2048
939 2048
940 2048
941 2048
942 2048
943 2048
944 2048
945 2048
946 2048
947 2048
948 2048
949 2048
950 2048
951 2048
952 2048
953 2048
954 2048
955 2048
956 2048
957 2048
958 2048
959 2048
960 2048
961 2048
962 2048
963 2048
964 2048
965 2048
966 2048
967 2048
968 2048
969 2048
970 2048
971 2048
972 2048
973 2048
974 2048
975 2048
976 2048
977 2048
978 2048
979 2048
980 2048
981 2048
982 2048
983 2048
984 2048
985 2048
986 2048
987 2048
988 2048
989 2048
990 2048
991 2048
992 2048
993 2048
994 2048
995 2048
996 2048
997 2048
998 2048
999 2048
1000 2048
1001 2048
1002 2048
1003 2048
1004 2048
1005 2048
1006 2048
1007 2048
1008 2048
1009 2048
1010 2048
1011 2048
1012 2048
1013 2048
1014 2048
1015 2048
1016 2048
1017 2048
1018 2048
1019 2048
1020 2048
1021 2048
1022 2048
1023 2048
1024 2048
1025 2048
1026 2048
1027 2048
1028 2048
1029 2048
1030 2048
1031 2048
1032 2048
1033

1766 2048
1767 2048
1768 2048
1769 2048
1770 2048
1771 2048
1772 2048
1773 2048
1774 2048
1775 2048
1776 2048
1777 2048
1778 2048
1779 2048
1780 2048
1781 2048
1782 2048
1783 2048
1784 2048
1785 2048
1786 2048
1787 2048
1788 2048
1789 2048
1790 2048
1791 2048
1792 2048
1793 2048
1794 2048
1795 2048
1796 2048
1797 2048
1798 2048
1799 2048
1800 2048
1801 2048
1802 2048
1803 2048
1804 2048
1805 2048
1806 2048
1807 2048
1808 2048
1809 2048
1810 2048
1811 2048
1812 2048
1813 2048
1814 2048
1815 2048
1816 2048
1817 2048
1818 2048
1819 2048
1820 2048
1821 2048
1822 2048
1823 2048
1824 2048
1825 2048
1826 2048
1827 2048
1828 2048
1829 2048
1830 2048
1831 2048
1832 2048
1833 2048
1834 2048
1835 2048
1836 2048
1837 2048
1838 2048
1839 2048
1840 2048
1841 2048
1842 2048
1843 2048
1844 2048
1845 2048
1846 2048
1847 2048
1848 2048
1849 2048
1850 2048
1851 2048
1852 2048
1853 2048
1854 2048
1855 2048
1856 2048
1857 2048
1858 2048
1859 2048
1860 2048
1861 2048
1862 2048
1863 2048
1864 2048
1865 2048


path ./data/circle_255.npz tensor([99])
100 tensor(99)
path ./data/circle_255.npz tensor([100])
101 tensor(100)
path ./data/circle_255.npz tensor([101])
102 tensor(101)
path ./data/circle_255.npz tensor([102])
103 tensor(102)
path ./data/circle_255.npz tensor([103])
104 tensor(103)
path ./data/circle_255.npz tensor([104])
105 tensor(104)
path ./data/circle_255.npz tensor([105])
106 tensor(105)
path ./data/circle_255.npz tensor([106])
107 tensor(106)
path ./data/circle_255.npz tensor([107])
108 tensor(107)
path ./data/circle_255.npz tensor([108])
109 tensor(108)
path ./data/circle_255.npz tensor([109])
110 tensor(109)
path ./data/circle_255.npz tensor([110])
111 tensor(110)
path ./data/circle_255.npz tensor([111])
112 tensor(111)
path ./data/circle_255.npz tensor([112])
113 tensor(112)
path ./data/circle_255.npz tensor([113])
114 tensor(113)
path ./data/circle_255.npz tensor([114])
115 tensor(114)
path ./data/circle_255.npz tensor([115])
116 tensor(115)
path ./data/circle_255.npz tensor

path ./data/circle_255.npz tensor([243])
244 tensor(243)
path ./data/circle_255.npz tensor([244])
245 tensor(244)
path ./data/circle_255.npz tensor([245])
246 tensor(245)
path ./data/circle_255.npz tensor([246])
247 tensor(246)
path ./data/circle_255.npz tensor([247])
248 tensor(247)
path ./data/circle_255.npz tensor([248])
249 tensor(248)
path ./data/circle_255.npz tensor([249])
250 tensor(249)
path ./data/circle_255.npz tensor([250])
251 tensor(250)
path ./data/circle_255.npz tensor([251])
252 tensor(251)
path ./data/circle_255.npz tensor([252])
253 tensor(252)
path ./data/circle_255.npz tensor([253])
254 tensor(253)
path ./data/circle_255.npz tensor([254])
255 tensor(254)
path ./data/circle_255.npz tensor([255])


(torch.Size([1, 3, 512, 512]), torch.Size([1, 3, 512, 512]))

In [9]:
square_no = 3
blueprint = Blueprint(stl_path, 512)
print(blueprint)
torch.save(blueprint.paths, f'./data/circle_plane_{len(blueprint.paths)}.pth')
blueprint_path = './data/circle_{}.npz'.format(square_no)
# print(blueprint_path)
res = blueprint.grid_sample(square_no, blueprint_path)
res['points'].shape, res['normals'].shape

0 512
1 512
2 512
3 512
4 512
5 512
6 512
7 512
8 512
9 512
10 512
11 512
12 512
13 512
14 512
15 512
16 512
17 512
18 512
19 512
20 512
21 512
22 512
23 512
24 512
25 512
26 512
27 512
28 512
29 512
30 512
31 512
32 512
33 512
34 512
35 512
36 512
37 512
38 512
39 512
40 512
41 512
42 512
43 512
44 512
45 512
46 512
47 512
48 512
49 512
50 512
51 512
52 512
53 512
54 512
55 512
56 512
57 512
58 512
59 512
60 512
61 512
62 512
63 512
64 512
65 512
66 512
67 512
68 512
69 512
70 512
71 512
72 512
73 512
74 512
75 512
76 512
77 512
78 512
79 512
80 512
81 512
82 512
83 512
84 512
85 512
86 512
87 512
88 512
89 512
90 512
91 512
92 512
93 512
94 512
95 512
96 512
97 512
98 512
99 512
100 512
101 512
102 512
103 512
104 512
105 512
106 512
107 512
108 512
109 512
110 512
111 512
112 512
113 512
114 512
115 512
116 512
117 512
118 512
119 512
120 512
121 512
122 512
123 512
124 512
125 512
126 512
127 512
128 512
129 512
130 512
131 512
132 512
133 512
134 512
135 512
136 512
137 512
138 51

(torch.Size([1, 3, 8, 8]), torch.Size([1, 3, 8, 8]))

In [10]:
import trimesh

from src.utilities.util import make_faces

res = np.load('./data/circle_{}.npz'.format(square_no))
points = torch.tensor(res['points'])
_, _, w, h,  = points.shape
faces = make_faces(w, h)
faces.shape

b_vertices = points.squeeze().reshape(3, -1).t()
print(b_vertices.shape)

mesh = trimesh.Trimesh(vertices=b_vertices, faces=faces)
mesh.show()

mesh.export('./data/m{}.stl'.format(square_no));

torch.Size([64, 3])


In [ ]:
res['points'].shape, res['normals'].shape

In [ ]:
square_no = 1023
blueprint_path = './data/vezuvio{}.npz'.format(square_no)
# print(blueprint_path)
res = blueprint.grid_sample(square_no, blueprint_path)
res['points'].shape, res['normals'].shape

In [ ]:
res = np.load('./data/vezuvio127.npz')#['points']#.shape

In [ ]:
import trimesh

from src.utilities.util import make_faces

res = np.load('./data/vezuvio127.npz')
points = torch.tensor(res['points'])
_, _, w, h,  = points.shape
faces = make_faces(w, h)
faces.shape

b_vertices = points.squeeze().reshape(3, -1).t()
print(b_vertices.shape)

mesh = trimesh.Trimesh(vertices=b_vertices, faces=faces)
mesh.show()

mesh.export('./m3.stl');

In [ ]:
square_no = 1
blueprint_path = './data/blueprint_radial_{}.npz'.format((square_no + 1) * 2)
print(blueprint_path)
res = blueprint.grid_sample(square_no, blueprint_path)
res['points'].shape, res['normals'].shape

In [ ]:
square_no = 1
side = (square_no + 1) * 2
side

In [ ]:
square_no = 1
side = (square_no + 1) * 2
side

In [ ]:
square_no = 2
side = (square_no + 1) * 2
side

In [ ]:
# 01
# 1
0.5
# 0123
# 1
# 2
# 3
1.5
# 012345
2.5
def get_angle(x, y, square_no):
    side = (square_no + 1) * 2
    corr = (side -1) / 2    
    angle = atan2(y -corr, x -corr)
    return angle if angle >= 0 else angle + 2 * pi

get_angle(0, 0, 0)

In [ ]:
res['points']

In [ ]:
sq_no, square_no = 0, 1
sq_ids = shifted_square_indices(sq_no, square_no)   
sq_ids

In [ ]:
angle = get_angle(1, 2)
angle

In [ ]:
for lat_angle in  blueprint.paths.keys():
    print(lat_angle)

In [ ]:
keys = list(blueprint.paths.keys())
len(keys)

In [ ]:
len(keys[:-1])

In [ ]:
4098 % 4096

In [ ]:
def get_border_paths(angle, paths):
    assert angle >=0 and angle < 2 * pi
    angle_keys = list(paths.keys())
    for i, angle_key in enumerate(angle_keys):        
        if angle < angle_key:            
            return (paths[angle_keys[i % len(keys)]],
                    paths[angle_keys[(i+1) % len(keys)]])
    return (paths[angle_keys[0]], paths[angle_keys[-1]])

angle = get_angle(1, 2)
paths = blueprint.paths
path0, path1 = get_border_paths(angle, paths)
path0, path1

In [ ]:
angle = 6.291651496887207
path0, path1 = get_border_paths(angle, paths)
path0, path1

In [ ]:
angle = 0
path0, path1 = get_border_paths(angle, paths)
path0, path1

In [ ]:
angle = -0.1
path0, path1 = get_border_paths(angle, paths)
path0, path1

In [ ]:
pnt0, nrm0 = single_path_sample(path0, sq_no, square_no)
pnt1, nrm1 = single_path_sample(path1, sq_no, square_no)


In [ ]:
pnt0, nrm0

In [ ]:
pnt0, pnt1, (pnt0 + pnt1)/2

In [ ]:
import torch.nn.functional as F

In [ ]:
F.normalize(nrm0, dim=0)